In [1]:
# Import all libraries here
import pyspark
import numpy as np
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
import pyspark.ml.evaluation as evals
import pyspark.ml.tuning as tune
from pyspark.ml import Pipeline
from pyspark import SparkContext

In [34]:
# Initialise spark by creating an instance of the SparkContext class
sc = SparkContext.getOrCreate()
print(sc.version)

2.4.5


In [35]:
# Create a StringIndexer
carr_indexer = StringIndexer(inputCol="carrier", outputCol="carrier_index")

In [36]:
# Create a OneHotEncoder
carr_encoder = OneHotEncoder(inputCol="carrier_index", outputCol="carrier_fact")

In [37]:
# Sample code for Making the pipeline
# flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler])

In [ ]:
# Fit and transform the data
piped_data = flights_pipe.fit(model_data).transform(model_data)

# Split the data into training and test sets
training, test = piped_data.randomSplit([.6, .4])

In [38]:
# Sample code for making a VectorAssembler
vec_assembler = VectorAssembler(inputCols=["month", "air_time", "carrier_fact", "dest_fact", "plane_age"], outputCol="features")

In [39]:
# Create a LogisticRegression Estimator
lr = LogisticRegression()

In [40]:
# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

In [48]:
# Create the parameter grid
grid = tune.ParamGridBuilder()

# Add the hyperparameter
grid = grid.addGrid(lr.regParam, np.arange(0.01, .1, .01))
grid = grid.addGrid(lr.elasticNetParam, np.arange(0.01, 2.01, 1))

# Build the grid
grid = grid.build()

In [49]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lr,
               estimatorParamMaps=grid,
               evaluator=evaluator)

In [52]:
""" Fitting the Model, using the dataset "training" """

# Call lr.fit()
best_lr = lr.fit(training)

# Print best_lr
print(best_lr)

' Fitting the Model, using the dataset "training" '

In [1]:
""" Evaluating the Model """

# Use the model to predict the test set
test_results = best_lr.transform(test)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

' Evaluating the Model '